In [ ]:
# import the necessary packages
from imutils import face_utils
import numpy as np
import imutils
import dlib
import cv2
from imutils.video import VideoStream

In [ ]:
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [ ]:
# initialize the video stream and allow the camera sensor to warmup
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
# (H, W) = (None, None)

In [ ]:
# loop over the frames from the video stream
while True:
    # read the next frame from the video stream and resize it
    frame = vs.read()
    frame = imutils.resize(frame, width=500)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # dlib needs RGB image format.
    
    # detect faces in the grayscale image
    rects = detector(rgb, 1)
    
    mask = np.zeros(rgb.shape, np.uint8)

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(rgb, rect)
        shape = face_utils.shape_to_np(shape)

        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)], then draw the face bounding box
        (x, y, w, h) = face_utils.rect_to_bb(rect)
        cv2.rectangle(rgb, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # show the face number
        cv2.putText(rgb, "Face #{}".format(i + 1), (x - 10, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # loop over the (x, y)-coordinates for the facial landmarks
        # and draw them on the image
        for (x1, y1) in shape:
            cv2.circle(mask, (x1, y1), 1, (0, 0, 255), -1)
            cv2.circle(rgb, (x1, y1), 1, (0, 0, 255), -1)
    
    
   # show the output frame
    cv2.imshow("Frame", rgb)
    cv2.imshow("Filtered", mask)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()